In [ ]:
# # try:
# #     import fitz  # PyMuPDF goes by `fitz`
# # except ImportError:
# #     !pip install PyMuPDF
# #     import fitz  # Try the import again after installing
    
# # import ipywidgets as widgets
# # from IPython.display import display, clear_output
# # import pandas as pd
# # import fitz  # PyMuPDF
# # from datetime import datetime
# # import re  # Import the re module for regex operations

# import fitz  # PyMuPDF
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# import pandas as pd
# from datetime import datetime
# import re  # Import the re module for regex operations

# # Define the style for the widgets
# style = {'description_width': 'initial'}

# # Widget Definitions
# excel_file_input = widgets.Text(placeholder='Enter the full path to your Excel file', layout=widgets.Layout(width='50%'), style=style, description='Excel Source File Path:')
# pdf_template_input = widgets.Text(placeholder='Enter the full path to your PDF template', layout=widgets.Layout(width='50%'), style=style, description='PDF Template:')
# save_path_input = widgets.Text(placeholder='Enter the path where you want to save the updated PDFs', layout=widgets.Layout(width='50%'), style=style, description='Save Path:')
# Rep = widgets.Text(placeholder='Enter Rep Name', layout=widgets.Layout(width='50%'), style=style, description='Rep Name:')
# Rep_ID = widgets.Text(placeholder='Enter Rep ID', layout=widgets.Layout(width='50%'), style=style, description='Rep ID:')

# case_conversion_dropdown = widgets.Dropdown(
#     options=[('None', 'None'), ('Upper', 'Upper'), ('Lower', 'Lower')],
#     value='None',
#     description='Name Case:',
#     layout=widgets.Layout(width='50%'), style=style,
# )
# process_button = widgets.Button(description="Process PDFs")
# output = widgets.Output()

# # Function to remove double quotes from the start and end of a string
# def remove_quotes(string):
#     return string.strip("\"")

# # Function to apply case conversion based on the user's choice
# def apply_case_conversion(name, choice):
#     if choice == 'Upper':
#         return name.upper()
#     elif choice == 'Lower':
#         return name.lower()
#     # Return the name as is if choice is 'None' or any other value
#     return name

# # Event Handler for the Button Click
# def on_button_clicked(b):
#     with output:
#         clear_output()
#         print("Starting PDF processing...")
#         total_rows = 0
#         successful_completions = 0
#         errors_count = 0
#         duplicates_count = 0
#         try:
#             # Sanitize input paths
#             excel_path = remove_quotes(excel_file_input.value)
#             template_path = remove_quotes(pdf_template_input.value)
#             save_path = remove_quotes(save_path_input.value)
            
#             # Ensure save_path ends with a slash (or backslash, depending on the OS)
#             if not save_path.endswith(('/', '\\')):
#                 save_path += '/'
            
#             # Read the Excel file
#             df = pd.read_excel(excel_path)

#             # Check for the required column 'Full Name' in the Excel file
#             required_column = 'Full Name'
#             if required_column not in df.columns:
#                 raise ValueError(f"The Excel file does not contain the required column '{required_column}'.")
            
#            # Convert the 'Full Name' column to strings
#             df['Full Name'] = df['Full Name'].astype(str)
#             total_rows = len(df)  # Total number of rows to process 
            
#             # Format today's date
#             todays_date = datetime.today().strftime('%m-%d-%Y')
            
#             pdf_document = fitz.open(template_path)
            
            
#             # Check for required PDF fields
#             required_fields = ["Check Box1", "Enter your full legal name here", "Date1_af_date", "Enter Representative Signature here", "Enter Representative ID here"]
#             pdf_fields = [widget.field_name for page in pdf_document for widget in page.widgets()]
            
#             missing_fields = [field for field in required_fields if field not in pdf_fields]
#             if missing_fields:
#                 missing_fields_str = ", ".join(missing_fields)
#                 raise ValueError(f"The PDF template is missing the following required fields: {missing_fields_str}.")
            
            
#             pdf_document.close()
            
#             # Initialize the set and list to keep track of processed names and errors
#             processed_names_set = set()
#             processed_names_list = []
#             errors_list = []
            
#             # Get the user's choice for case conversion
#             case_choice = case_conversion_dropdown.value
            
#             # Process each row in the DataFrame
#             for index, row in df.iterrows():
#                 document_opened = False
                
#                 try:
#                     # Validate the 'Full Name' field
#                     full_name = row['Full Name']
#                     # Skipping explicit null check as it's redundant now
#                     if full_name.strip() == '' or full_name == 'nan':
#                         raise ValueError(f"Full Name is empty at row {index + 1}.")
#                     # Removed isinstance check since full_name is always a string now
#                     if len(full_name.strip()) < 5:
#                         raise ValueError(f"Full Name '{full_name}' is too short at row {index + 1}.")
#                     if not re.match("^[a-zA-Z ,.'-]+$", full_name):
#                         raise ValueError(f"Full Name '{full_name}' contains invalid characters at row {index + 1}.")

#                     full_name = apply_case_conversion(full_name, case_choice)
                    
#                     # Check for duplicate names
#                     if full_name in processed_names_set:
#                         print(f"Duplicate detected for '{full_name}' at row {index + 1}, skipped duplicate record.")
#                         duplicates_count += 1
#                         continue  # Skip processing this duplicate name
                    
                    
#                     pdf_document = fitz.open(template_path)
#                     document_opened = True  # This line is necessary to indicate the document is now open
                    
                    
#                     # Fill in the form fields
#                     for page in pdf_document:
#                         widgets = page.widgets()
#                         for widget in widgets:
#                             if widget.field_name == "Check Box1":
#                                 widget.field_value = widget.on_state()
#                                 widget.update()
#                             elif widget.field_name == "Enter your full legal name here":
#                                 widget.field_value = full_name
#                                 widget.update()
#                             elif widget.field_name in ["Date1_af_date", "Date2_af_date"]:
#                                 widget.field_value = todays_date
#                                 widget.update()
#                             elif widget.field_name == "Enter Representative Signature here":
#                                 widget.field_value = Rep.value
#                                 widget.update()
#                             elif widget.field_name == "Enter Representative ID here":
#                                 widget.field_value = Rep_ID.value
#                                 widget.update()
                    
#                     new_filename = f"TransferExceptionForm_{todays_date}_{full_name.replace(' ', '_')}.pdf"
#                     updated_pdf_path = f"{save_path}{new_filename}"
                    
#                     pdf_document.save(updated_pdf_path)
#                     successful_completions += 1
#                     #print(f"PDF saved as {new_filename}.")
                    
#                     processed_names_set.add(full_name)
#                     processed_names_list.append(full_name)
                
#                 except Exception as e:
#                     errors_count += 1
#                     error_message = f"Error processing '{full_name if 'full_name' in locals() else 'Unknown'}' at row {index + 1}: {str(e)}"
#                     errors_list.append(error_message)
#                     print(error_message)
                
#                 finally:
#                     # Only attempt to close the document if it has been opened
#                     if document_opened:
                        
#                         pdf_document.close()
                        
            
#             print("\nCompleted processing all rows.")
#             # Display summary of processing
#             print(f"\nSummary of processing: {successful_completions} out of {total_rows} rows were completed successfully.")
#             print(f"Errors encountered: {errors_count}. Duplicates skipped: {duplicates_count}.")
            
#             if processed_names_list:
#                 pd.DataFrame(processed_names_list, columns=['Processed Names']).to_csv(f"{save_path}processed_names_{todays_date}.csv", index=False)
#                 print(f"\nProcessed names saved to {save_path}processed_names_{todays_date}.csv")
#             if errors_list:
#                 pd.DataFrame(errors_list, columns=['Errors']).to_csv(f"{save_path}errors_{todays_date}.csv", index=False)
#                 print(f"\nErrors saved to {save_path}errors_{todays_date}.csv")
                
#         except Exception as e:
#             print(f"An error occurred: {e}")
        
# # Bind the button click event to the handler
# process_button.on_click(on_button_clicked)

# # Display the UI components
# display(excel_file_input, pdf_template_input, save_path_input, case_conversion_dropdown, Rep, Rep_ID, process_button, output)


In [3]:
# # Ensure necessary imports are done at the beginning of your script.
# import fitz  # PyMuPDF
# import ipywidgets as widgets
# from IPython.display import display, clear_output, HTML
# import pandas as pd
# import io
# from zipfile import ZipFile
# import base64

# # Define the style for the widgets.
# style = {'description_width': 'initial'}

# # Widget Definitions.
# excel_file_upload = widgets.FileUpload(accept='.xlsx', multiple=False,  description="Upload Excel", layout=widgets.Layout(width='50%'), style=style)
# pdf_template_upload = widgets.FileUpload(accept='.pdf', multiple=False, description="Upload PDF Template", layout=widgets.Layout(width='50%'), style=style)
# case_conversion_dropdown = widgets.Dropdown(
#     options=[('None', 'None'), ('Upper', 'Upper'), ('Lower', 'Lower')],
#     value='None',
#     description='Name Case:',
#     layout=widgets.Layout(width='50%'), 
#     style=style,
# )
# process_button = widgets.Button(description="Process PDFs")
# output = widgets.Output()

# # Event Handler for the Button Click.
# def on_button_clicked(b):
#     with output:
#         clear_output()
#         print("Starting PDF processing...")
#         processed_names_list = []
#         errors_list = []
        
#         try:
#             # Read the uploaded Excel file.
#             excel_content = next(iter(excel_file_upload.value.values()))['content']
#             df = pd.read_excel(io.BytesIO(excel_content))

#             # Read the uploaded PDF template.
#             pdf_template_content = next(iter(pdf_template_upload.value.values()))['content']
#             pdf_document = fitz.open(stream=io.BytesIO(pdf_template_content), filetype="pdf")

#             # Case choice for name conversion.
#             case_choice = case_conversion_dropdown.value

#             # Create a ZIP file to store PDFs.
#             zip_buffer = io.BytesIO()
#             with ZipFile(zip_buffer, 'w') as zip_file:
#                 # Process each row in the DataFrame and create a PDF.
#                 for index, row in df.iterrows():
#                     try:
#                         full_name = apply_case_conversion(row['Full Name'], case_choice)
#                         processed_names_list.append({'Full Name': full_name})
#                         # ... (logic to process each row and generate PDFs)
#                         # Here, you should include the logic to fill the PDF and save it to the ZIP.
#                     except Exception as e:
#                         errors_list.append({'Error': str(e), 'Row': index + 1})
#                         print(f"Error processing row {index + 1}: {str(e)}")

#                 # Optionally, convert lists to DataFrames and add them to the ZIP as CSV files.
#                 # This part is left as an exercise to implement as needed.

#             # Prepare the ZIP file for download.
#             zip_buffer.seek(0)
#             b64 = base64.b64encode(zip_buffer.read()).decode()
#             href = f'<a href="data:file/zip;base64,{b64}" download="processed_files.zip">Download zip file</a>'
#             display(HTML(href))
#         except Exception as e:
#             print(f"An error occurred: {e}")

# # Bind the button click event to the handler and display UI components.
# process_button.on_click(on_button_clicked)
# display(excel_file_upload, pdf_template_upload, case_conversion_dropdown, process_button, output)

FileUpload(value={}, accept='.xlsx', description='Upload Excel')

FileUpload(value={}, accept='.pdf', description='Upload PDF Template')

Dropdown(description='Name Case:', layout=Layout(width='50%'), options=(('None', 'None'), ('Upper', 'Upper'), …

Button(description='Process PDFs', style=ButtonStyle())

Output()

In [21]:
# try:
#     import fitz  # PyMuPDF goes by `fitz`
# except ImportError:
#     !pip install PyMuPDF
#     import fitz  # Try the import again after installing
    
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# import pandas as pd
# import fitz  # PyMuPDF
# from datetime import datetime
# import re  # Import the re module for regex operations

import fitz  # PyMuPDF
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import io
from zipfile import ZipFile
import base64
from datetime import datetime
import re  # Import the re module for regex operations

# Define the style for the widgets
style = {'description_width': 'initial'}

# Widget Definitions
excel_file_upload = widgets.FileUpload(accept='.xlsx', multiple=False,  description="Upload Excel", layout=widgets.Layout(width='50%'), style=style)
pdf_template_upload = widgets.FileUpload(accept='.pdf', multiple=False, description="Upload PDF Template", layout=widgets.Layout(width='50%'), style=style)
save_path_input = widgets.Text(placeholder='Enter the path where you want to save the updated PDFs', layout=widgets.Layout(width='50%'), style=style, description='Save Path:')
Rep = widgets.Text(placeholder='Enter Rep Name', layout=widgets.Layout(width='50%'), style=style, description='Rep Name:')
Rep_ID = widgets.Text(placeholder='Enter Rep ID', layout=widgets.Layout(width='50%'), style=style, description='Rep ID:')

case_conversion_dropdown = widgets.Dropdown(
    options=[('None', 'None'), ('Upper', 'Upper'), ('Lower', 'Lower')],
    value='None',
    description='Name Case:',
    layout=widgets.Layout(width='50%'), style=style,
)
process_button = widgets.Button(description="Process PDFs")
output = widgets.Output()


# Function to remove double quotes from the start and end of a string
def remove_quotes(string):
    return string.strip("\"")

# Function to apply case conversion based on the user's choice
def apply_case_conversion(name, choice):
    if choice == 'Upper':
        return name.upper()
    elif choice == 'Lower':
        return name.lower()
    # Return the name as is if choice is 'None' or any other value
    return name

# Event Handler for the Button Click
def on_button_clicked(b):
    with output:
        clear_output()
        print("Starting PDF processing...")
        total_rows = 0
        successful_completions = 0
        errors_count = 0
        duplicates_count = 0

        try:    
            # Read the uploaded Excel file.
            print("trying to define excel path")
            uploaded_file = excel_file_upload.value[0]
            excel_path = uploaded_file['content']
            #excel_path = next(iter(excel_file_upload.value.values()))['content']
            print("successfully define excel path")
            print("trying to define pdf path")
            template_path = pdf_template_upload.value[0]
            print("successfully define excel path")
            print("rying to optn pdf path")
            pdf_document = fitz.open(stream=io.BytesIO(template_path), filetype="pdf")
            print("opened pdf")
            
            # Read the Excel file
            print("trying to read excel")
            df = pd.read_excel(io.BytesIO(excel_path))
            print("successfully read excel")
            # Check for the required column 'Full Name' in the Excel file
            required_column = 'Full Name'
            if required_column not in df.columns:
                raise ValueError(f"The Excel file does not contain the required column '{required_column}'.")
            
           # Convert the 'Full Name' column to strings
            df['Full Name'] = df['Full Name'].astype(str)
            total_rows = len(df)  # Total number of rows to process 
            
            # Format today's date
            todays_date = datetime.today().strftime('%m-%d-%Y')
            
            # Check for required PDF fields
            required_fields = ["Check Box1", "Enter your full legal name here", "Date1_af_date", "Enter Representative Signature here", "Enter Representative ID here"]
            pdf_fields = [widget.field_name for page in pdf_document for widget in page.widgets()]
            
            missing_fields = [field for field in required_fields if field not in pdf_fields]
            if missing_fields:
                missing_fields_str = ", ".join(missing_fields)
                raise ValueError(f"The PDF template is missing the following required fields: {missing_fields_str}.")
            
            
            #pdf_document.close()
            
            # Define the sanitize_filename function if you haven't already
            def sanitize_filename(name):
                # A simple example of sanitization, replace this with your actual function
                return "".join(c for c in name if c.isalnum() or c in " _-").rstrip()

            # Before the for loop
            processed_names_set = set()
            processed_names_list = []
            errors_list = []
            
            # Get the user's choice for case conversion
            case_choice = case_conversion_dropdown.value
            
            # Create a ZIP file to store PDFs.
            
            zip_buffer = io.BytesIO()
            with ZipFile(zip_buffer, 'w') as zip_file:
                # Process each row in the DataFrame and create a PDF.
            
                # Process each row in the DataFrame
                for index, row in df.iterrows():
                    document_opened = False

                    try:
                        # Validate the 'Full Name' field
                        full_name = row['Full Name']
                        # Skipping explicit null check as it's redundant now
                        if full_name.strip() == '' or full_name == 'nan':
                            raise ValueError(f"Full Name is empty at row {index + 1}.")
                        # Removed isinstance check since full_name is always a string now
                        if len(full_name.strip()) < 5:
                            raise ValueError(f"Full Name '{full_name}' is too short at row {index + 1}.")
                        if not re.match("^[a-zA-Z ,.'-]+$", full_name):
                            raise ValueError(f"Full Name '{full_name}' contains invalid characters at row {index + 1}.")

                        full_name = apply_case_conversion(full_name, case_choice)

                        # Check for duplicate names
                        if full_name in processed_names_set:
                            print(f"Duplicate detected for '{full_name}' at row {index + 1}, skipped duplicate record.")
                            duplicates_count += 1
                            continue  # Skip processing this duplicate name


                        sanitized_full_name = sanitize_filename(full_name)
                        
                        # Create the PDF in-memory
                        pdf_bytes = io.BytesIO()
                        # Open the PDF from the bytes stream, fill it in, save to pdf_bytes
                        pdf_document = fitz.open(stream=io.BytesIO(template_path), filetype="pdf")
                        
                        document_opened = True  # This line is necessary to indicate the document is now open
                        
                        

                        # Fill in the form fields
                        for page in pdf_document:
                            widgets = page.widgets()
                            for widget in widgets:
                                if widget.field_name == "Check Box1":
                                    widget.field_value = widget.on_state()
                                    widget.update()
                                elif widget.field_name == "Enter your full legal name here":
                                    widget.field_value = full_name
                                    widget.update()
                                elif widget.field_name in ["Date1_af_date", "Date2_af_date"]:
                                    widget.field_value = todays_date
                                    widget.update()
                                elif widget.field_name == "Enter Representative Signature here":
                                    widget.field_value = Rep.value
                                    widget.update()
                                elif widget.field_name == "Enter Representative ID here":
                                    widget.field_value = Rep_ID.value
                                    widget.update()

                        
                        pdf_document.save(pdf_bytes)
                        pdf_document.close()

                        # Add the in-memory PDF to the ZIP file
                        pdf_bytes.seek(0)
                        zip_file.writestr(f"{sanitized_full_name}.pdf", pdf_bytes.read())

                        # Add to processed names list
                        processed_names_list.append(sanitized_full_name)
                        successful_completions += 1

                    except Exception as e:
                        errors_count += 1
                        error_message = f"Error processing '{full_name if 'full_name' in locals() else 'Unknown'}' at row {index + 1}: {str(e)}"
                        errors_list.append(error_message)
                        print(error_message)

#                     finally:
#                         # Only attempt to close the document if it has been opened
#                         if document_opened:
#                             print("x")
# #                             pdf_document.close()
                        
            
            print("\nCompleted processing all rows.")
            # Display summary of processing
            print(f"\nSummary of processing: {successful_completions} out of {total_rows} rows were completed successfully.")
            print(f"Errors encountered: {errors_count}. Duplicates skipped: {duplicates_count}.")
            
            # Now write the CSVs for processed names and errors to the ZIP buffer
            processed_names_df = pd.DataFrame(processed_names_list, columns=['Processed Names'])
            errors_df = pd.DataFrame(errors_list, columns=['Errors'])

            with ZipFile(zip_buffer, 'w') as zip_file:
                # Save processed names CSV
                processed_names_csv = io.StringIO()
                processed_names_df.to_csv(processed_names_csv, index=False)
                processed_names_csv.seek(0)
                zip_file.writestr('processed_names.csv', processed_names_csv.getvalue())

                # Save errors CSV
                errors_csv = io.StringIO()
                errors_df.to_csv(errors_csv, index=False)
                errors_csv.seek(0)
                zip_file.writestr('errors.csv', errors_csv.getvalue())

            # Prepare the ZIP file for download
            zip_buffer.seek(0)
            b64 = base64.b64encode(zip_buffer.read()).decode()
            download_filename = f"Processed_Files_{datetime.today().strftime('%m-%d-%Y')}.zip"
            href = f'<a href="data:application/zip;base64,{b64}" download="{download_filename}">Download zip file</a>'
            display(HTML(href))
            
        except Exception as e:
            print(f"An error occurred: {e}")

# Bind the button click event to the handler and display UI components.
process_button.on_click(on_button_clicked)
display(excel_file_upload, pdf_template_upload, Rep, Rep_ID, case_conversion_dropdown, process_button, output)


FileUpload(value={}, accept='.xlsx', description='Upload Excel', layout=Layout(width='50%'))

FileUpload(value={}, accept='.pdf', description='Upload PDF Template', layout=Layout(width='50%'))

Text(value='', description='Rep Name:', layout=Layout(width='50%'), placeholder='Enter Rep Name', style=Descri…

Text(value='', description='Rep ID:', layout=Layout(width='50%'), placeholder='Enter Rep ID', style=Descriptio…

Dropdown(description='Name Case:', layout=Layout(width='50%'), options=(('None', 'None'), ('Upper', 'Upper'), …

Button(description='Process PDFs', style=ButtonStyle())

Output()